<a href="https://colab.research.google.com/github/Sudesh-chandra/AmazonRecommendationEngine/blob/main/Another_copy_of_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import plotly.express as px
from surprise import SVD, NMF

In [ ]:
from google.colab import files
uploded = files.upload()


Saving Amazon_Products.csv to Amazon_Products.csv


In [ ]:
path = "Amazon_Products.csv"
df = pd.read_csv(path,low_memory=False)
df.head()

,uniq_id,product_name,manufacturer,price,number_available_in_stock,number_of_reviews,number_of_answered_questions,average_review_rating,amazon_category_and_sub_category,description,...,Unnamed: 885,Unnamed: 886,Unnamed: 887,Unnamed: 888,Unnamed: 889,Unnamed: 890,Unnamed: 891,Unnamed: 892,Unnamed: 893,Unnamed: 894
0,eac7efa5dbd3d667f26eb3d3ab504464,Hornby 2014 Catalogue,Hornby,£3.42,5 new,15,1,4.9 out of 5 stars,Hobbies > Model Trains & Railway Sets > Rail V...,Product Description Hornby 2014 Catalogue Box ...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,b17540ef7e86e461d37f3ae58b7b72ac,FunkyBuys® Large Christmas Holiday Express Fes...,FunkyBuys,£16.99,NaN,2,1,4.5 out of 5 stars,Hobbies > Model Trains & Railway Sets > Rail V...,Size Name:Large FunkyBuys® Large Christmas Hol...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,348f344247b0c1a935b1223072ef9d8a,CLASSIC TOY TRAIN SET TRACK CARRIAGES LIGHT EN...,ccf,£9.99,2 new,17,2,3.9 out of 5 stars,Hobbies > Model Trains & Railway Sets > Rail V...,BIG CLASSIC TOY TRAIN SET TRACK CARRIAGE LIGHT...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,e12b92dbb8eaee78b22965d2a9bbbd9f,HORNBY Coach R4410A BR Hawksworth Corridor 3rd,Hornby,£39.99,NaN,1,2,5.0 out of 5 stars,Hobbies > Model Trains & Railway Sets > Rail V...,Hornby 00 Gauge BR Hawksworth 3rd Class W 2107...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,e33a9adeed5f36840ccc227db4682a36,Hornby 00 Gauge 0-4-0 Gildenlow Salt Co. Steam...,Hornby,£32.19,NaN,3,2,4.7 out of 5 stars,Hobbies > Model Trains & Railway Sets > Rail V...,Product Description Hornby RailRoad 0-4-0 Gild...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
df.shape

(10004, 895)

In [ ]:
df.isnull().sum()

uniq_id                          2
product_name                     2
manufacturer                     9
price                         1437
number_available_in_stock     2502
                             ...  
Unnamed: 890                 10002
Unnamed: 891                 10002
Unnamed: 892                 10002
Unnamed: 893                 10003
Unnamed: 894                 10003
Length: 895, dtype: int64

In [ ]:
perc = 70.0
min_count =  int(((100 - perc)/100) * df.shape[1] + 1)
mod_df = df.dropna(axis = 1, thresh = min_count)

In [ ]:
mod_df.shape

(10004, 14)

In [ ]:
mod_df.columns

Index(['uniq_id', 'product_name', 'manufacturer', 'price',
       'number_available_in_stock', 'number_of_reviews',
       'number_of_answered_questions', 'average_review_rating',
       'amazon_category_and_sub_category', 'description',
       'product_information', 'product_description',
       'items_customers_buy_after_viewing_this_item',
       'customer_questions_and_answers'],
      dtype='object')

In [ ]:
mod_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10004 entries, 0 to 10003
Data columns (total 14 columns):
 #   Column                                       Non-Null Count  Dtype 
---  ------                                       --------------  ----- 
 0   uniq_id                                      10002 non-null  object
 1   product_name                                 10002 non-null  object
 2   manufacturer                                 9995 non-null   object
 3   price                                        8567 non-null   object
 4   number_available_in_stock                    7502 non-null   object
 5   number_of_reviews                            9984 non-null   object
 6   number_of_answered_questions                 9237 non-null   object
 7   average_review_rating                        9984 non-null   object
 8   amazon_category_and_sub_category             9312 non-null   object
 9   description                                  9351 non-null   object
 10  product_in

In [ ]:
mod_df.dropna(subset=['uniq_id'], inplace = True)


<ipython-input-11-4eb203380abb>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mod_df.dropna(subset=['uniq_id'], inplace = True)


In [ ]:
code_len = mod_df["uniq_id"].map(lambda code: len(code))
code_len.unique()

array([32,  1])

In [ ]:
# Here it seems some problem is present. let us check it
for i in mod_df["uniq_id"]:
    if len(i) == 1:
        print(i)


}
}


In [ ]:
# here it seems that } is the unwanted entry. Thus we need to remove those rows
mod_df[mod_df["uniq_id"] == "}"]

,uniq_id,product_name,manufacturer,price,number_available_in_stock,number_of_reviews,number_of_answered_questions,average_review_rating,amazon_category_and_sub_category,description,product_information,product_description,items_customers_buy_after_viewing_this_item,customer_questions_and_answers
2003,},"{""""text"""":""""Digital & Prime Music""""","panelKey"""":""""Mp3Panel""""}","{""""subtext"""":""""A great selection of apps and g...","text"""":""""Appstore for Android""""","subtextKey"""":""""android-tagline""""","panelKey"""":""""AndroidPanel""""}","{""""text"""":""""Amazon Cloud Drive""""","panelKey"""":""""CloudDrivePanel""""}","panelKey"""":""""KindleReaderPanel""""}","{""""text"""":""""Fire Tablets""""","panelKey"""":""""KindleFireTabletPanel""""}","{""""text"""":""""Fire TV""""","panelKey"""":""""FireTvPanel""""}"
2005,},"{""""text"""":""""Digital & Prime Music""""","panelKey"""":""""Mp3Panel""""}","{""""subtext"""":""""A great selection of apps and g...","text"""":""""Appstore for Android""""","subtextKey"""":""""android-tagline""""","panelKey"""":""""AndroidPanel""""}","{""""text"""":""""Amazon Cloud Drive""""","panelKey"""":""""CloudDrivePanel""""}","panelKey"""":""""KindleReaderPanel""""}","{""""text"""":""""Fire Tablets""""","panelKey"""":""""KindleFireTabletPanel""""}","{""""text"""":""""Fire TV""""","panelKey"""":""""FireTvPanel""""}"


In [ ]:
# let us drop these rows
mod_df.drop([2003,2005], inplace = True)

<ipython-input-15-ba306f0b378e>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mod_df.drop([2003,2005], inplace = True)


In [ ]:
mod_df[mod_df['number_available_in_stock'].str.contains("used", na = False)].head(2)

,uniq_id,product_name,manufacturer,price,number_available_in_stock,number_of_reviews,number_of_answered_questions,average_review_rating,amazon_category_and_sub_category,description,product_information,product_description,items_customers_buy_after_viewing_this_item,customer_questions_and_answers
17,e9ef14a0253f074343b5441540d8471f,Chuggington Interactive Wash and Fuel Set with...,Chuggington,NaN,1 used,2,1,4.0 out of 5 stars,Hobbies > Model Trains & Railway Sets > Rail V...,Product Description The Chuggington Interactiv...,Technical Details Item Weight998 g Product Dim...,Product Description The Chuggington Interactiv...,http://www.amazon.co.uk/Chuggington | http://w...,NaN
161,a8d29a2c05bb402335f29f433eb20beb,Hornby R322 OO Scale Class 8F 2-8-0 Steam Loco...,Hornby,NaN,1 used,1,2,5.0 out of 5 stars,Hobbies > Model Trains & Railway Sets > Rail V...,An OO scale model of Stanier's heavy freight s...,Technical Details Manufacturer recommended age...,An OO scale model of Stanier's heavy freight s...,NaN,NaN


In [ ]:
mod_df = mod_df[mod_df["uniq_id"] != "}"]


In [ ]:
item = ["new", "used"]
mod_df["item_type"] = mod_df['number_available_in_stock'].str.extract(f'({"|".join(item)})')

In [ ]:
mod_df["item_type"].unique()

array(['new', nan, 'used'], dtype=object)

In [ ]:
mod_df["price_new"] = mod_df["price"].str.split(" ", expand = True).get(0).str.split("£", expand = True).get(1)
# check if the value is in range of thousand
mod_df[mod_df.price_new.str.contains(',', na=False)]
# remove the comma sign
mod_df["price_new"] = mod_df["price_new"].str.replace(",", "")

In [ ]:
import numpy as np
import re

# Convert columns to string type if they are not already
mod_df["number_available_in_stock"] = mod_df["number_available_in_stock"].astype(str)
mod_df["average_review_rating"] = mod_df["average_review_rating"].astype(str)

# Define function to extract numbers from strings
def extract_numeric(string):
    match = re.search(r'\d+(\.\d+)?', string)
    return float(match.group()) if match else np.nan

# Extract numbers from "number_available_in_stock" column
mod_df["number_available_in_stock"] = mod_df["number_available_in_stock"].apply(extract_numeric)

# Extract numbers from "average_review_rating" column
mod_df["average_review_rating(out of 5)"] = mod_df["average_review_rating"].apply(extract_numeric)

# Handling NaN values in both columns
mod_df["number_available_in_stock"].fillna(0, inplace=True)  # Replace NaN with 0 or any other suitable value
mod_df["average_review_rating(out of 5)"].fillna(0, inplace=True)  # Replace NaN with 0 or any other suitable value


In [ ]:
mod_df["average_review_rating(out of 5)"] = mod_df["average_review_rating(out of 5)"].astype(float)
mod_df["price_new"] = mod_df["price_new"].astype(float)

In [ ]:
import pandas as pd
import numpy as np

# Remove commas from the "number_of_reviews" column
mod_df["number_of_reviews"] = mod_df["number_of_reviews"].str.replace(",", "")

# Convert values to numeric, coercing errors to NaN
mod_df["number_of_reviews"] = pd.to_numeric(mod_df["number_of_reviews"], errors='coerce')

# Check for rows with NaN values in the "number_of_reviews" column
invalid_rows = mod_df[pd.to_numeric(mod_df["number_of_reviews"], errors='coerce').isnull()]

# Remove rows with NaN values from the DataFrame
mod_df = mod_df.drop(invalid_rows.index)

# Change the type of column to float
mod_df["number_of_reviews"] = mod_df["number_of_reviews"].astype(float)


In [ ]:
mod_df["number_of_answered_questions"] = mod_df["number_of_answered_questions"].astype(float)

In [ ]:
mod_df.drop(["price", "average_review_rating"],  axis=1, inplace = True)

In [ ]:
mod_df.head(2)

,uniq_id,product_name,manufacturer,number_available_in_stock,number_of_reviews,number_of_answered_questions,amazon_category_and_sub_category,description,product_information,product_description,items_customers_buy_after_viewing_this_item,customer_questions_and_answers,item_type,price_new,average_review_rating(out of 5)
0,eac7efa5dbd3d667f26eb3d3ab504464,Hornby 2014 Catalogue,Hornby,5.0,15.0,1.0,Hobbies > Model Trains & Railway Sets > Rail V...,Product Description Hornby 2014 Catalogue Box ...,Technical Details Item Weight640 g Product Dim...,Product Description Hornby 2014 Catalogue Box ...,http://www.amazon.co.uk/Hornby-R8150-Catalogue...,Does this catalogue detail all the previous Ho...,new,3.42,4.9
1,b17540ef7e86e461d37f3ae58b7b72ac,FunkyBuys® Large Christmas Holiday Express Fes...,FunkyBuys,0.0,2.0,1.0,Hobbies > Model Trains & Railway Sets > Rail V...,Size Name:Large FunkyBuys® Large Christmas Hol...,Technical Details Manufacturer recommended age...,Size Name:Large FunkyBuys® Large Christmas Hol...,http://www.amazon.co.uk/Christmas-Holiday-Expr...,can you turn off sounds // hi no you cant turn...,NaN,16.99,4.5


In [ ]:
mod_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9982 entries, 0 to 10003
Data columns (total 15 columns):
 #   Column                                       Non-Null Count  Dtype  
---  ------                                       --------------  -----  
 0   uniq_id                                      9982 non-null   object 
 1   product_name                                 9982 non-null   object 
 2   manufacturer                                 9975 non-null   object 
 3   number_available_in_stock                    9982 non-null   float64
 4   number_of_reviews                            9982 non-null   float64
 5   number_of_answered_questions                 9218 non-null   float64
 6   amazon_category_and_sub_category             9298 non-null   object 
 7   description                                  9335 non-null   object 
 8   product_information                          9923 non-null   object 
 9   product_description                          9334 non-null   object 
 10 

In [ ]:
x = mod_df.isnull().sum()

fig = px.bar(x, orientation = "h",  text_auto='.2s',
            color_discrete_sequence= ["#ff6b00"] * len(x))
fig.update_layout(
    title="<b>Missing Value Count</b>",
    xaxis_title="Total missing values",
    yaxis_title="Column Names",
    plot_bgcolor = "#ECECEC",
    showlegend=False
)
fig.show()

In [ ]:
missing_price = mod_df[mod_df["price_new"].isnull()]
missing_price.head(2)

,uniq_id,product_name,manufacturer,number_available_in_stock,number_of_reviews,number_of_answered_questions,amazon_category_and_sub_category,description,product_information,product_description,items_customers_buy_after_viewing_this_item,customer_questions_and_answers,item_type,price_new,average_review_rating(out of 5)
9,5afbaf65680c9f378af5b3a3ae22427e,Learning Curve Chuggington Interactive Chatsworth,Chuggington,1.0,8.0,1.0,Hobbies > Model Trains & Railway Sets > Rail V...,Product Description An amazingly Interactive C...,Technical Details Item Weight150 g Product Dim...,Product Description An amazingly Interactive C...,http://www.amazon.co.uk/Chuggington | http://w...,NaN,new,NaN,4.8
14,c68c3ae3b0ea3146beae99f3d4a6997c,Kato 3060-2 EF65 500 (F Model) Electric Locomo...,Kato,18.0,1.0,1.0,Hobbies > Model Trains & Railway Sets > Rail V...,Suitable for the following scale(s): N Scale,Technical Details Item Weight150 g Product Dim...,Suitable for the following scale(s): N Scale,NaN,NaN,new,NaN,5.0


In [ ]:
df = mod_df.dropna(subset=['price_new'])
df.head()

,uniq_id,product_name,manufacturer,number_available_in_stock,number_of_reviews,number_of_answered_questions,amazon_category_and_sub_category,description,product_information,product_description,items_customers_buy_after_viewing_this_item,customer_questions_and_answers,item_type,price_new,average_review_rating(out of 5)
0,eac7efa5dbd3d667f26eb3d3ab504464,Hornby 2014 Catalogue,Hornby,5.0,15.0,1.0,Hobbies > Model Trains & Railway Sets > Rail V...,Product Description Hornby 2014 Catalogue Box ...,Technical Details Item Weight640 g Product Dim...,Product Description Hornby 2014 Catalogue Box ...,http://www.amazon.co.uk/Hornby-R8150-Catalogue...,Does this catalogue detail all the previous Ho...,new,3.42,4.9
1,b17540ef7e86e461d37f3ae58b7b72ac,FunkyBuys® Large Christmas Holiday Express Fes...,FunkyBuys,0.0,2.0,1.0,Hobbies > Model Trains & Railway Sets > Rail V...,Size Name:Large FunkyBuys® Large Christmas Hol...,Technical Details Manufacturer recommended age...,Size Name:Large FunkyBuys® Large Christmas Hol...,http://www.amazon.co.uk/Christmas-Holiday-Expr...,can you turn off sounds // hi no you cant turn...,NaN,16.99,4.5
2,348f344247b0c1a935b1223072ef9d8a,CLASSIC TOY TRAIN SET TRACK CARRIAGES LIGHT EN...,ccf,2.0,17.0,2.0,Hobbies > Model Trains & Railway Sets > Rail V...,BIG CLASSIC TOY TRAIN SET TRACK CARRIAGE LIGHT...,Technical Details Manufacturer recommended age...,BIG CLASSIC TOY TRAIN SET TRACK CARRIAGE LIGHT...,http://www.amazon.co.uk/Train-With-Tracks-Batt...,What is the gauge of the track // Hi Paul.Trut...,new,9.99,3.9
3,e12b92dbb8eaee78b22965d2a9bbbd9f,HORNBY Coach R4410A BR Hawksworth Corridor 3rd,Hornby,0.0,1.0,2.0,Hobbies > Model Trains & Railway Sets > Rail V...,Hornby 00 Gauge BR Hawksworth 3rd Class W 2107...,Technical Details Item Weight259 g Product Dim...,Hornby 00 Gauge BR Hawksworth 3rd Class W 2107...,NaN,NaN,NaN,39.99,5.0
4,e33a9adeed5f36840ccc227db4682a36,Hornby 00 Gauge 0-4-0 Gildenlow Salt Co. Steam...,Hornby,0.0,3.0,2.0,Hobbies > Model Trains & Railway Sets > Rail V...,Product Description Hornby RailRoad 0-4-0 Gild...,Technical Details Item Weight159 g Product Dim...,Product Description Hornby RailRoad 0-4-0 Gild...,http://www.amazon.co.uk/Hornby-R2672-RailRoad-...,NaN,NaN,32.19,4.7


In [ ]:
df[["price_new", "manufacturer"]].groupby("manufacturer").max().sort_values(by = "price_new",
                                                                    ascending = False)

,price_new
manufacturer,
Bowen Designs,2439.92
DJI,995.11
Sideshow,719.95
AUTOart,648.95
Bushiroad,629.95
...,...
k2-accessories Acrylic Plastic Beads,0.90
Card Concept,0.83
unisex party bag fillers,0.60


In [ ]:
df[df["price_new"] == df["price_new"].max()]

,uniq_id,product_name,manufacturer,number_available_in_stock,number_of_reviews,number_of_answered_questions,amazon_category_and_sub_category,description,product_information,product_description,items_customers_buy_after_viewing_this_item,customer_questions_and_answers,item_type,price_new,average_review_rating(out of 5)
7098,75d96583894e50412ba5c0161375fef5,Bowen Designs - Marvel Statue Thor Classic Act...,Bowen Designs,2.0,1.0,1.0,Hobbies > Model Trains & Railway Sets > Access...,Sideshow Collectibles is proud to present the ...,Technical Details Item Weight6 Kg Product Dime...,Sideshow Collectibles is proud to present the ...,NaN,NaN,new,2439.92,4.0


In [ ]:
df[df["price_new"] == df["price_new"].min()]

,uniq_id,product_name,manufacturer,number_available_in_stock,number_of_reviews,number_of_answered_questions,amazon_category_and_sub_category,description,product_information,product_description,items_customers_buy_after_viewing_this_item,customer_questions_and_answers,item_type,price_new,average_review_rating(out of 5)
6366,30950ba413fbae88b190efb5c7237fc2,Happy 18th Birthday Foil Banner - 365cm,Partyrama,0.0,1.0,1.0,"Party Supplies > Banners, Stickers & Confetti ...",Happy 18th Birthday Foil Banner - Sold Single....,Technical Details Manufacturer recommended age...,Happy 18th Birthday Foil Banner - Sold Single....,http://www.amazon.co.uk/18th-Happy-Birthday-La...,NaN,NaN,0.01,5.0


In [ ]:
values = df["manufacturer"].value_counts().keys().tolist()[:10]
counts = df["manufacturer"].value_counts().tolist()[:10]

In [ ]:
fig = px.bar(df, y = counts, x = values,
            color_discrete_sequence = ["#EC2781"] * len(df))


fig.update_layout(
                 plot_bgcolor = "#ECECEC",
                  yaxis_title = "Count",
                xaxis_title = "Name of Manufacturers",
                  title = "<b>Popular Manufacturers Category</b>"
                 )
fig.show()

In [ ]:
df_list = []
for i in values:
    x = df[df["manufacturer"] == i]
    df_list.append(x)
frame = pd.concat(df_list)
frame.head(2)

,uniq_id,product_name,manufacturer,number_available_in_stock,number_of_reviews,number_of_answered_questions,amazon_category_and_sub_category,description,product_information,product_description,items_customers_buy_after_viewing_this_item,customer_questions_and_answers,item_type,price_new,average_review_rating(out of 5)
804,d994e8cb4d0b15683da91652901d9895,Oxford Diecast Anglia Police Panda 76105003,Oxford Diecast,4.0,4.0,11.0,Die-Cast & Toy Vehicles > Toy Vehicles & Acces...,"The Ford Anglia , appeared in 1959 and when it...",Technical Details Item Weight18 g Product Dime...,"The Ford Anglia , appeared in 1959 and when it...",http://www.amazon.co.uk/Cararama-Anglia-Sunbur...,NaN,new,4.75,4.8
830,8f4a9ace7bb4ae6e6d8dda96d94bd332,Oxford Diecast Evening News Metal Model,Oxford Diecast,5.0,1.0,1.0,Die-Cast & Toy Vehicles > Toy Vehicles & Acces...,Product Description In the early 1960s the mos...,Technical Details Item Weight82 g Product Dime...,Product Description In the early 1960s the mos...,http://www.amazon.co.uk/Omnibus-RM075-OMNIBUS-...,NaN,new,10.95,5.0


In [ ]:
frame[["manufacturer", "average_review_rating(out of 5)"]].groupby("manufacturer").mean().sort_values(by = "average_review_rating(out of 5)",
                                                ascending = False)

,average_review_rating(out of 5)
manufacturer,
MyTinyWorld,4.923656
Oxford Diecast,4.790789
Corgi,4.783333
The Puppet Company,4.759804
Star Wars,4.738710
LEGO,4.736232
Playmobil,4.726496
Disney,4.687500
Mattel,4.661039


In [ ]:
print("The average rating: ",frame["average_review_rating(out of 5)"].unique())

# After processing our data we have significantly reduced the size of the dataframe.
# Also the rating are now 4 or greater.
# Let us now check new average price ### check above before processing to compare.
print("The new average price: ", frame["price_new"].mean())

The average rating:  [4.8 5.  4.5 4.  4.4 4.7 4.6 4.3 4.9 4.1 4.2 3.6 2.3]
The new average price:  20.96527962085308


In [ ]:
print("Does this new data have used items as well: ", frame["item_type"].value_counts())
print("There are still used item in our top selected dataframe. Let us check these items.")
# still we have some used item. let us check these used item
frame_used = frame.loc[frame["item_type"] == "used"]
frame_used.head(2)

Does this new data have used items as well:  new     837
used     10
Name: item_type, dtype: int64
There are still used item in our top selected dataframe. Let us check these items.


,uniq_id,product_name,manufacturer,number_available_in_stock,number_of_reviews,number_of_answered_questions,amazon_category_and_sub_category,description,product_information,product_description,items_customers_buy_after_viewing_this_item,customer_questions_and_answers,item_type,price_new,average_review_rating(out of 5)
3537,373fac5c5e9297c17e578b2330b72ba7,Oxford Diecast Black/Burgundy (Rothchild) Humb...,Oxford Diecast,1.0,1.0,1.0,Die-Cast & Toy Vehicles > Toy Vehicles & Acces...,Another highly detailed luxury car joins the O...,Technical Details Item Weight136 g Product Dim...,Another highly detailed luxury car joins the O...,NaN,NaN,used,20.95,5.0
2496,66c72659d135b5968afdd3c3206dcd2c,LEGO Bionicle 8570 Gali Nuva,LEGO,3.0,2.0,1.0,NaN,"Although she personifies the deepest, darkest ...",Technical Details Item Weight200 g Product Dim...,"Although she personifies the deepest, darkest ...",http://www.amazon.co.uk/LEGO-Bionicle-70789-On...,NaN,used,51.15,5.0


In [ ]:
frame_used["average_review_rating(out of 5)"].mean()

4.5

In [ ]:
import plotly.figure_factory as ff
x = frame["price_new"]
hist_data = [x]
group_labels = ['price']

fig = ff.create_distplot(hist_data, group_labels, show_rug = False,
                        colors=["#ffd514"])
fig.update_layout(
                 plot_bgcolor = "#ECECEC",
                  title = "<b>Price Distribution of Data</b>"
                 )

fig.show()

In [ ]:
frame.price_new.describe()

count    1055.000000
mean       20.965280
std        27.891015
min         0.960000
25%         7.225000
50%        12.990000
75%        24.860000
max       374.960000
Name: price_new, dtype: float64

In [ ]:
fig = px.box(frame, "price_new")
fig.update_layout(
                 plot_bgcolor = "#ECECEC",
                  title = "<b>Price Data Distribution</b>",
                 xaxis_title = "Price of Products"
                 )
fig.show()

In [ ]:
Q1 = 5.95
Q2 = 10.37
Q3 = 20.12
IQR = Q3 - Q1
outlier1 = (Q1 - 1.5 * IQR)
outlier2 = (Q3 + 1.5 * IQR)
print("outlier1: ", outlier1)
print("outlier2: ", outlier2)

outlier1:  -15.305000000000003
outlier2:  41.375


In [ ]:
outlier_price = []

for i in frame.price_new:
    if i < outlier1 or i > outlier2:
        outlier_price.append("outlier")
    elif i > outlier1 or i < outlier2:
        outlier_price.append("normal")

frame["outlier_price"] = outlier_price

In [ ]:
fig = px.pie(frame, names = frame["outlier_price"], color = frame["outlier_price"],
             color_discrete_map={'normal': '#2377a4', 'outlier': '#ffd514'})

fig.update_layout(title = "<b>Distribution of Outlier</b>")

fig.show()

In [ ]:
frame_outlier = frame.loc[frame["outlier_price"] == "outlier"].head()
frame_outlier

,uniq_id,product_name,manufacturer,number_available_in_stock,number_of_reviews,number_of_answered_questions,amazon_category_and_sub_category,description,product_information,product_description,items_customers_buy_after_viewing_this_item,customer_questions_and_answers,item_type,price_new,average_review_rating(out of 5),outlier_price
614,1cde07d3e45fdfb1b1966368d3b7a522,Lego City 2824: Advent Calendar 2010,LEGO,19.0,40.0,6.0,NaN,"The holidays are coming to LEGO City, one gift...",Technical Details Item Weight440 g Product Dim...,"The holidays are coming to LEGO City, one gift...",http://www.amazon.co.uk/LEGO-City-60099-Advent...,NaN,new,43.87,4.7,outlier
627,15e03b806b3d90bced2186049c4f3613,LEGO Star Wars 75018: Jek-14's Stealth Starfig...,LEGO,16.0,30.0,1.0,Characters & Brands > Star Wars > Toys,Product Description LEGO® Star Wars™ Jek-14’s ...,Technical Details Item Weight907 g Product Dim...,Product Description LEGO® Star Wars™ Jek-14’s ...,http://www.amazon.co.uk/Star-Wars-75102-Fighte...,NaN,new,68.87,4.8,outlier
667,eeadf21a379a4c1d47eaf674b5ba321b,LEGO City 7907: Advent Calendar,LEGO,7.0,3.0,1.0,NaN,NaN,Technical Details Item Weight340 g Product Dim...,NaN,http://www.amazon.co.uk/LEGO-City-60063-Advent...,NaN,new,61.14,4.7,outlier
720,eaf710937250fc9a964f96aa4c834395,Lego City 7553: Advent Calendar,LEGO,11.0,67.0,1.0,NaN,Lego 7553 - City - Advent Calendar It's Christ...,Technical Details Item Weight440 g Product Dim...,Lego 7553 - City - Advent Calendar It's Christ...,http://www.amazon.co.uk/LEGO-City-60099-Advent...,NaN,new,49.90,4.6,outlier
739,0960eb8ceb6c621609763012f912d22f,Lego Kingdoms Advent Calendar 7952,LEGO,10.0,14.0,1.0,NaN,Celebrate 24 December days full of medieval bu...,Technical Details Item Weight363 g Product Dim...,Celebrate 24 December days full of medieval bu...,http://www.amazon.co.uk/LEGO-City-60099-Advent...,NaN,new,69.99,4.8,outlier


In [ ]:
print("Manufacturers with outlier price: ", frame_outlier.manufacturer.value_counts())

Manufacturers with outlier price:  LEGO    5
Name: manufacturer, dtype: int64


In [ ]:
print("Item_type with outlier price: ", frame_outlier.item_type.unique())

Item_type with outlier price:  ['new']


In [ ]:
frame_outlier[frame_outlier["item_type"] == "used"]

,uniq_id,product_name,manufacturer,number_available_in_stock,number_of_reviews,number_of_answered_questions,amazon_category_and_sub_category,description,product_information,product_description,items_customers_buy_after_viewing_this_item,customer_questions_and_answers,item_type,price_new,average_review_rating(out of 5),outlier_price


In [ ]:
fig = px.violin(frame, "average_review_rating(out of 5)",
               color_discrete_sequence = ["#FFBF00"] * len(frame))
fig.update_layout(
                 plot_bgcolor = "#ECECEC",
                  xaxis_title = "Rating",
                  title = "<b>Rating Distribution of the Popular Products</b>"
                 )
fig.show()

In [ ]:
fig = px.histogram(frame, "number_of_reviews",
                  color_discrete_sequence = ["#8B4000"] * len(frame))
fig.update_layout(
                 plot_bgcolor = "#ECECEC",
                  xaxis_title = "Number of Reviews",
                  title = "<b>Number of Reviews Distribution</b>"
                 )
fig.show()

In [ ]:
print("Number of null values: ",frame['number_of_reviews'].isnull().sum())
# it seems that with high end products people love to leave a review

Number of null values:  0


In [ ]:
frame.head(2)

,uniq_id,product_name,manufacturer,number_available_in_stock,number_of_reviews,number_of_answered_questions,amazon_category_and_sub_category,description,product_information,product_description,items_customers_buy_after_viewing_this_item,customer_questions_and_answers,item_type,price_new,average_review_rating(out of 5),outlier_price
804,d994e8cb4d0b15683da91652901d9895,Oxford Diecast Anglia Police Panda 76105003,Oxford Diecast,4.0,4.0,11.0,Die-Cast & Toy Vehicles > Toy Vehicles & Acces...,"The Ford Anglia , appeared in 1959 and when it...",Technical Details Item Weight18 g Product Dime...,"The Ford Anglia , appeared in 1959 and when it...",http://www.amazon.co.uk/Cararama-Anglia-Sunbur...,NaN,new,4.75,4.8,normal
830,8f4a9ace7bb4ae6e6d8dda96d94bd332,Oxford Diecast Evening News Metal Model,Oxford Diecast,5.0,1.0,1.0,Die-Cast & Toy Vehicles > Toy Vehicles & Acces...,Product Description In the early 1960s the mos...,Technical Details Item Weight82 g Product Dime...,Product Description In the early 1960s the mos...,http://www.amazon.co.uk/Omnibus-RM075-OMNIBUS-...,NaN,new,10.95,5.0,normal


In [ ]:
fig = px.scatter(frame, x="number_available_in_stock", y="number_of_reviews",
                 trendline="ols")

fig.update_layout(title = "<b>Relationship between the number of reviews and product stock</b>",
                 plot_bgcolor = "#ECECEC",
                 yaxis_title = "Number of reviews",
                 xaxis_title = "Number available in stock")

fig.show()

In [ ]:
fig = px.histogram(frame, "number_available_in_stock",
                  color_discrete_sequence = ["#C04000"] * len(frame))
fig.update_layout(
                 plot_bgcolor = "#ECECEC",
                  xaxis_title = "Products Available in Stocks",
                  title = "<b>Number of Products Available in Stocks</b>"
                 )
fig.show()

In [ ]:
mod_df.columns

Index(['uniq_id', 'product_name', 'manufacturer', 'number_available_in_stock',
       'number_of_reviews', 'number_of_answered_questions',
       'amazon_category_and_sub_category', 'description',
       'product_information', 'product_description',
       'items_customers_buy_after_viewing_this_item',
       'customer_questions_and_answers', 'item_type', 'price_new',
       'average_review_rating(out of 5)'],
      dtype='object')

In [ ]:
ratings = pd.DataFrame(df.groupby('product_name')['average_review_rating(out of 5)'].mean())
ratings['ratings_count'] = pd.DataFrame(df.groupby('product_name')['average_review_rating(out of 5)'].count())
ratings['ratings_average'] = pd.DataFrame(df.groupby('product_name')['average_review_rating(out of 5)'].mean())
ratings=ratings[0:1978]
ratings.head(10)

,average_review_rating(out of 5),ratings_count,ratings_average
product_name,,,
"""5 daisy"" spacer bead 8mm x 3mm. Pack of 15",5.0,1,5.0
"""ALICE IN WONDERLAND QUEEN OF HEARTS RABBIT HERALD"" BADGE BUTTON PIN (1inch/25mm diameter) LEWIS CARROLL",4.0,1,4.0
"""COMMUNIST SOVIET UNION FLAG"" RETRO EIGHTIES BADGE BUTTON PIN (1inch/25mm diameter)",5.0,1,5.0
"""JAMES DEAN"" FIFTIES HOLLYWOOD BADGE BUTTON PIN (1inch/25mm diameter) 1950s",5.0,1,5.0
"""JoJo's Bizarre Adventure"" 6. Kakyoin Noriaki (Hirohiko Araki Specify Color) Part Three Super Figure moving (reproduction) (japan import)",4.7,1,4.7
"""JoJo's Bizarre Adventure"" Part 5 54 Trish Una Super Figure moving (japan import)",5.0,1,5.0
"""MOD"" CLASSIC RETRO SIXTIES MOD BADGE BUTTON PIN (1inch/25mm diameter)",5.0,1,5.0
"""Relaxing with Friends"" Alzheimer's & Dementia Therapeutic Activity, 12 Pc. Puzzle",5.0,1,5.0
"""Star Wars"" [Cosplay] Hasbro lightsaber ""Level 2 / Electronic"" Qui-Gon Jinn (japan import)",4.7,1,4.7


Collaborative filtering model like SVD

In [ ]:
from surprise import Dataset, Reader, SVD
from surprise.model_selection import train_test_split
from surprise import accuracy

reader = Reader(rating_scale=(1, 5))
data = Dataset.load_from_df(mod_df[['uniq_id', 'product_name', 'average_review_rating(out of 5)']], reader)

CF_trainset, CF_testset = train_test_split(data, test_size=0.2, random_state=42)

CF_model = SVD()

CF_model.fit(CF_trainset)

CF_predictions = CF_model.test(CF_testset)

In [ ]:
uniq_id = '060a2cc5828457b08bd436c9211c2633'

user_rated_items = mod_df[mod_df['uniq_id'] == uniq_id]['product_name']

all_items = mod_df['product_name'].unique()
items_to_predict = [item for item in all_items if item not in user_rated_items]

recommendations = []
for item_id in items_to_predict:
    predicted_rating = CF_model.predict(uniq_id, item_id).est
    recommendations.append((item_id, predicted_rating))

recommendations.sort(key=lambda x: x[1], reverse=True)

top_n_recommendations = recommendations[:10]

print("Top Recommended Items for User", uniq_id)
for item_id, predicted_rating in top_n_recommendations:
    print("Item:", item_id, "Predicted Rating:", predicted_rating)

Top Recommended Items for User 060a2cc5828457b08bd436c9211c2633
Item: 100 six sided dice, 14mm, random colours Predicted Rating: 4.763036504751572
Item: Hot Wheels Batman 2015: Batman Begins Batmobile Predicted Rating: 4.757315368252541
Item: Fisher-Price Imaginext Raptor Predicted Rating: 4.757056953156699
Item: Plastoy Tubo Mythology Tube Predicted Rating: 4.756408559582822
Item: 9" I Love You Heart Me to You Bear Predicted Rating: 4.756374332082486
Item: Hama My First Midi Predicted Rating: 4.755998192877391
Item: Pokemon Card Game Jungle Booster Pack [Toy] Predicted Rating: 4.755813220839072
Item: My Little Pony Friendship Magic Collection - Plush Doll 6 Pieces Doll Set 6.5" (Rarity, Pinkie Pie, Applejack, Fluttershy, Rainbow Dash and Twilight Sparkle) Predicted Rating: 4.755609119991669
Item: Ty Angeline - Happy Valentine's Day Doll with Gift Box Predicted Rating: 4.755302585554947
Item: Tobar 1:12 Scale "Aprilia Rsv4 Factory" Diecast Model Bike Kit Predicted Rating: 4.75513365802

Matrix Factorization

In [ ]:
from surprise import Dataset, Reader
from surprise.model_selection import train_test_split
from surprise import SVD
from surprise import accuracy

reader = Reader(rating_scale=(0, 5))
data = Dataset.load_from_df(mod_df[['uniq_id', 'product_name', 'average_review_rating(out of 5)']], reader)

MF_trainset, MF_testset = train_test_split(data, test_size=0.2, random_state=42)

MF_model = SVD()

MF_model.fit(MF_trainset)

MF_predictions = MF_model.test(MF_testset)

rmse = accuracy.rmse(MF_predictions)
accuracy= accuracy.mse(MF_predictions)

RMSE: 0.3679
MSE: 0.1353


In [ ]:
def get_recommendations(uniq_id, model):
    all_item_ids = mod_df['product_name'].unique()

    predicted_ratings = [(item_id, model.predict(uniq_id, item_id).est) for item_id in all_item_ids]

    predicted_ratings.sort(key=lambda x: x[1], reverse=True)

    recommendations = [item_id for item_id, _ in predicted_ratings]

    recommendations = recommendations[:10]

    return recommendations

In [ ]:
uniq_id = '060a2cc5828457b08bd436c9211c2633'

all_recommendations = get_recommendations(uniq_id, MF_model)

print("All Recommended Items for User", uniq_id)
for i in all_recommendations:
  print(i)

All Recommended Items for User 060a2cc5828457b08bd436c9211c2633
Polyhedral Dice
IdentiPen Dual Point Marker
Siku 1:32 Claas Axion 850 Tractor
Mensa Assess your Personality
Kotobukiya Alien: Big Chap Chopsticks
Quantum Mechanix Super 8: The Argus Cube Replica
Hanayama Duet Puzzle
GE Entertainment Death Note Plush Toy - 7" Ryuk (GE-7050)
SOFT TOY collie puppy 22cm
Miniature Yatzy dice game. Ideal Christmas Cracker filler


Content Based Filtering

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
vectorizer = TfidfVectorizer()
mod_df['amazon_category_and_sub_category'].isnull().sum()
mod_df['amazon_category_and_sub_category'].fillna("", inplace=True)
tfidf_matrix = vectorizer.fit_transform(mod_df['amazon_category_and_sub_category'])

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
similarities = cosine_similarity(tfidf_matrix)

In [ ]:
similarities_result = pd.DataFrame(similarities, index=mod_df['product_name'], columns=mod_df['product_name'])
similarities_result.sample(10, random_state=42).sample(4, axis=1, random_state=4242)

product_name,"Welly ""Aston Martin DB9 Coupe"" Car",MATCHBOX MODELS OF YESTERYEAR.Grand Prix.Y-14/1935 `R.I.B`.,Sword Art Online SK series Asuna (non-scale PVC painted action figure) (japan import),6-sided Dice: Translucent Red
product_name,,,,
"Perler Pearl Stripe Beads (1000-Piece, Raspberry)",0.000000,0.000000,0.000000,0.000000
Tobar 00435 Wooden Domino Race,0.000000,0.000000,0.000000,0.379067
Hasbro A7902 Guardians of The Galaxy Big Blastin Rocket Raccoon Action Figure,0.000000,0.000000,0.265901,0.000000
Dolls House Miniature 1:12 Scale Pub Bar Accessory Wooden Wine Beer Barrel,0.034509,0.040223,0.000000,0.000000
Meccano Eiffel Tower Construction Set,0.000000,0.000000,0.000000,0.000000
Imaginext® Space Station,0.000000,0.000000,0.000000,0.000000
Alps Glacier Express (Add-On 4-Car Set) (Model Train),0.165403,0.128527,0.000000,0.000000
HAMA BEADS RABBIT SINGLE PEGBOARD,0.000000,0.000000,0.000000,0.000000
Scalextric C3083 PRO Performance - Chevrolet Impala SS 1:32 Scale Slot Car Kit,0.023629,0.027541,0.000000,0.000000


In [ ]:
def content_based(product_name:str, recommendation_limit:int=10, similarities_data:pd.DataFrame=similarities_result, book_data:pd.DataFrame=mod_df) -> pd.DataFrame:


    index = similarities_data.loc[:,product_name].to_numpy().argpartition(range(-1, -(recommendation_limit+2), -1))
    most_similiar = similarities_data.columns[index[-1:-(recommendation_limit+2):-1]]
    most_similiar = most_similiar.drop(product_name, errors='ignore')
    recomendations = pd.DataFrame(most_similiar).merge(book_data).head(recommendation_limit)

    return product_name, recomendations.sort_values(['average_review_rating(out of 5)'], ascending=False)

In [ ]:
sample_product = mod_df.sample(5, random_state=37)
sample_product

,uniq_id,product_name,manufacturer,number_available_in_stock,number_of_reviews,number_of_answered_questions,amazon_category_and_sub_category,description,product_information,product_description,items_customers_buy_after_viewing_this_item,customer_questions_and_answers,item_type,price_new,average_review_rating(out of 5)
9571,9cf7a36a7ae5abca5945688d64759215,NECA Dr. Zaius Planet of the Apes Series 1 - 7...,NECA,25.0,2.0,1.0,Hobbies > Collectible Figures & Memorabilia > ...,"From the beloved 1968 film, Planet of the Apes...",Technical Details Item Weight227 g Product Dim...,"From the beloved 1968 film, Planet of the Apes...",http://www.amazon.co.uk/Cornelius-Planet-Serie...,NaN,new,14.81,4.5
5396,dd28f2a6d001f85eb8fc644e45141842,Bicycle Pressers Deck by ellusionist,Ellusionist,2.0,1.0,6.0,Games > Card Games,"Pulled from the 60's, this is one of Don and R...",Technical Details Item Weight91 g Product Dime...,"Pulled from the 60's, this is one of Don and R...",http://www.amazon.co.uk/Bicycle-Guardians-Deck...,NaN,new,8.99,4.0
2015,417b371e7bcddd70d14dfd77f7c41674,Micro Chargers - Electronic Micro Racing Cars ...,Micro Chargers,0.0,3.0,1.0,"Hobbies > Slot Cars, Race Tracks & Accessories...",Micro Chargers Light Racers Hyper Dome. 1 x Hy...,Technical Details Item Weight499 g Product Dim...,Micro Chargers Light Racers Hyper Dome. 1 x Hy...,http://www.amazon.co.uk/Moose-Toys-Micro-Charg...,NaN,NaN,NaN,5.0
2169,97adfc9e84af437024f071d29c05aaec,Carrera GO!!! 6.3 m Lap Record! Red Bull and F...,Carrera 'Go',1.0,2.0,1.0,"Hobbies > Slot Cars, Race Tracks & Accessories...",Product Description The ultimate Race Set for ...,Technical Details Item Weight3.1 Kg Product Di...,Product Description The ultimate Race Set for ...,http://www.amazon.co.uk/Scalextric | http://ww...,NaN,new,NaN,4.5
4776,4edfb78b974b27f2319d2c4d18d37d18,Trading Card Storage Wallets Sleeves Album Pag...,Max Protect,3.0,31.0,3.0,Hobbies > Trading Cards & Accessories > Storag...,"Suitable for Standard Sized Trading cards , su...",Technical Details BrandMax Protect Item Weight...,"Suitable for Standard Sized Trading cards , su...",http://www.amazon.co.uk/Ultra-Pro-Collectors |...,NaN,new,2.69,4.7


In [ ]:
name_1, recommendations_1 = content_based('Most Popular Inkwear! Pack of Temp Tattoos', 5)
print('Product Name:', name_1)
display(recommendations_1)

Product Name: Most Popular Inkwear! Pack of Temp Tattoos


,product_name,uniq_id,manufacturer,number_available_in_stock,number_of_reviews,number_of_answered_questions,amazon_category_and_sub_category,description,product_information,product_description,items_customers_buy_after_viewing_this_item,customer_questions_and_answers,item_type,price_new,average_review_rating(out of 5)
0,Halloween Glitter Tattoo Kit - HYPOALLERGENIC ...,94079878290d7c2303114ae1be521b27,GlitZGlam,2.0,1.0,1.0,Fancy Dress > Accessories > Temporary Tattoos,Halloween Body Art kit set includes a collecti...,Technical Details Item Weight91 g Product Dime...,Halloween Body Art kit set includes a collecti...,http://www.amazon.co.uk/Snazaroo-Face-Paint-St...,NaN,new,NaN,5.0
1,Gold Eyelash Bindi ✮ DESIGNER BINDIS ✮ Festiva...,0f3b4c2689eac502e8715415f118f410,KARIZMA,0.0,1.0,1.0,Fancy Dress > Accessories > Temporary Tattoos,From holidays in Ibiza to music festivals and ...,Technical Details Manufacturer recommended age...,From holidays in Ibiza to music festivals and ...,NaN,NaN,NaN,3.50,5.0
3,Ultimate Temporary Tattoo Kit / Gift Set,f6e497916b5107a29c726b4af11c363b,Glitter Tattoo Kits,0.0,5.0,1.0,Fancy Dress > Accessories > Temporary Tattoos,Ultimate Temporary Tattoo Kit / Gift Set This ...,Technical Details Item Weight458 g Product Dim...,Ultimate Temporary Tattoo Kit / Gift Set This ...,http://www.amazon.co.uk/Girls-Body-Art-Glitter...,NaN,NaN,26.95,4.8
2,Glitter Tattoo Kit New GlitZGlam with 6 LARGE ...,02d0fa4040741b3686512ff039f27b21,GlitZGlam,3.0,14.0,1.0,Fancy Dress > Accessories > Temporary Tattoos,GlitZGlam Glitter Tattoo Kit is a best seller ...,Technical Details Item Weight200 g Product Dim...,GlitZGlam Glitter Tattoo Kit is a best seller ...,NaN,NaN,used,NaN,4.5
4,Pack Of 24 St Patricks Day Glitter Tattoo Sten...,278d5cf30f9887d04b2416a1ed4cbace,Glitter Tattoo Stencils,0.0,1.0,1.0,Fancy Dress > Accessories > Temporary Tattoos,Top of the morning to you! A stencil pack to m...,Technical Details Item Weight27 g Product Dime...,Top of the morning to you! A stencil pack to m...,http://www.amazon.co.uk/Ireland-TP0061-Tempora...,NaN,NaN,3.85,4.0


In [ ]:
name_2, recommendations_2 = content_based("ILOVEFANCYDRESS® LADIES FLAPPER FANCY DRESS COSTUME 1920'S FRINGE DRESS IN BLACK PINK OR RED WITH MATCHING FEATHER SEQUIN HEADPIECE 20'S CHARLSTON", 5)
print('Product Name:', name_2)
display(recommendations_2)

Product Name: ILOVEFANCYDRESS® LADIES FLAPPER FANCY DRESS COSTUME 1920'S FRINGE DRESS IN BLACK PINK OR RED WITH MATCHING FEATHER SEQUIN HEADPIECE 20'S CHARLSTON


,product_name,uniq_id,manufacturer,number_available_in_stock,number_of_reviews,number_of_answered_questions,amazon_category_and_sub_category,description,product_information,product_description,items_customers_buy_after_viewing_this_item,customer_questions_and_answers,item_type,price_new,average_review_rating(out of 5)
0,1940's-WW2-Wartime FLORAL PRINT EVACUEE DRESS ...,43dd6c77f417aacaf76134226d794a5c,CRAZYLADIES COSTUMES,0.0,1.0,1.0,Fancy Dress > Accessories > Sets & Kits,Customer Services 01978 661046,Technical Details Manufacturer recommended age...,Customer Services 01978 661046,http://www.amazon.co.uk/Evacuee-Girl-Childrens...,NaN,NaN,15.99,5.0
2,Gothic Nurse Uniform Female Fancy Dress Costum...,4adb8ac394661ffd94b78c8319a9ecae,Parties Unwrapped,2.0,1.0,1.0,Fancy Dress > Accessories > Sets & Kits,Gothic Nurse uniform fancy dress costume compl...,Technical Details Manufacturer recommended age...,Gothic Nurse uniform fancy dress costume compl...,NaN,NaN,new,11.99,5.0
1,Mamma Mia 70s Disco Blue Ladies Fancy Dress Ha...,8c879f9b14c7ca6945666bd18810be67,Wicked Costumes,7.0,11.0,1.0,Fancy Dress > Accessories > Sets & Kits,XL Ladies Mamma Mia Costume for Abba 70s 80s F...,Technical Details Product Dimensions43.6 x 26....,XL Ladies Mamma Mia Costume for Abba 70s 80s F...,NaN,don't understand the sizing I am assize 12 14 ...,new,22.03,4.3
3,TAVERN WENCH ADULT FANCY DRESS COSTUME LADIES ...,e429e677677a754741ef6c75141cb4f2,Sancto,0.0,6.0,1.0,Fancy Dress > Accessories > Sets & Kits,NaN,NaN,NaN,NaN,NaN,NaN,22.39,4.3
4,TAVERN WENCH ADULT FANCY DRESS COSTUME LADIES ...,2f846a8dc2327760795dd11b4621d5c5,Sancto,0.0,6.0,2.0,Fancy Dress > Costumes > Adults,NaN,NaN,NaN,NaN,NaN,NaN,22.00,4.3


In [ ]:
import pandas as pd
from surprise import Dataset, Reader, KNNWithMeans
from surprise.model_selection import train_test_split


reader = Reader(rating_scale=(1, 5))

surprise_data = Dataset.load_from_df(mod_df[['uniq_id', 'product_name', 'average_review_rating(out of 5)']], reader)

trainset, testset = train_test_split(surprise_data, test_size=0.2)

model = KNNWithMeans(k=5, sim_options={'name': 'pearson_baseline', 'user_based': False})
model.fit(trainset)

def get_recommendations(uniq_id, model=model):
    all_item_ids = list(mod_df['product_name'].unique())

    rated_items = mod_df[mod_df['uniq_id'] == uniq_id]['product_name'].tolist()
    items_to_predict = [item_id for item_id in all_item_ids if item_id not in rated_items]

    predictions = [model.predict(uniq_id, item_id) for item_id in items_to_predict]

    sorted_predictions = sorted(predictions, key=lambda x: x.est, reverse=True)

    top_recommendations = [(pred.iid, pred.est) for pred in sorted_predictions[:10]]
    return top_recommendations



Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.


In [ ]:
#Recommending using the unique ID of the Reviews of Customers.

uniq_id = "d479d878b358113d5b7eb05bf3a86bb5"
recommendations = get_recommendations(uniq_id)
print("Top Recommendations for User", uniq_id)
for item_id, estimated_rating in recommendations:
    print(item_id, "Estimated Rating:", estimated_rating)

Top Recommendations for User d479d878b358113d5b7eb05bf3a86bb5
HORNBY Coach R4410A BR Hawksworth Corridor 3rd Estimated Rating: 5
20pcs Model Garden Light Double Heads Lamppost Scale 1:100 Estimated Rating: 5
Hornby Gauge Western Express Digital Train Set with eLink and TTS Loco Train Set Estimated Rating: 5
Hornby Gauge Railroad Mosley Tarmacadam Locomotive Estimated Rating: 5
Hornby 00 Gauge 253mm Weathered Paviland Grange Steam Locomotive Train Model Estimated Rating: 5
Glacier Express of N gauge 10-1219 Alps [UNESCO paint color] 7 Car Set Estimated Rating: 5
Bachmann 31-588 Freightliner Class 70 005 Powerhaul Diesel Weathered Estimated Rating: 5
Preiser 30495 Horse Drawn Wedding Coach (Closed) Estimated Rating: 5
Preiser 30414 Horse Drawn Liquid Manure Wagon Estimated Rating: 5
Bachmann Class A2 60534 'Irish Elegance' BR Lined Green Early Emblem Estimated Rating: 5


In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
from surprise import Reader, Dataset, SVD
import random


dtypes = {col: 'str' for col in df.columns}
df = pd.read_csv("Amazon_Products.csv", dtype=dtypes,low_memory=False)

df['amazon_category_and_sub_category'] = df['amazon_category_and_sub_category'].fillna("")  # Handle missing values
df['combined_features'] = df['product_name'] + ' ' + df['amazon_category_and_sub_category']

df = df.dropna(subset=['combined_features'])


tfidf_vectorizer = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf_vectorizer.fit_transform(df['combined_features'])


cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)


def preprocess_ratings(rating):
    try:
        return float(rating.split()[0])
    except:
        return None

df['average_review_rating'] = df['average_review_rating'].map(preprocess_ratings)

# Hybrid recommendation function
def hybrid_recommendations(product_name, svd_model, similarity_matrix=cosine_sim, df=df, top_n=10):
    content_based_recs = get_content_based_recommendations(product_name, similarity_matrix, df, top_n)
    collaborative_filtering_recs = get_collaborative_filtering_recommendations(product_name, svd_model, df, top_n)
    hybrid_recs = content_based_recs.tolist() + collaborative_filtering_recs
    return random.sample(hybrid_recs, min(len(hybrid_recs), top_n))

#  content-based recommendations
def get_content_based_recommendations(product_name, similarity_matrix=cosine_sim, df=df, top_n=10):
    idx = df.index[df['product_name'] == product_name].tolist()[0]
    sim_scores = list(enumerate(similarity_matrix[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:top_n+1]
    product_indices = [i[0] for i in sim_scores]
    return df.iloc[product_indices]['product_name']

def get_collaborative_filtering_recommendations(product_name, svd_model, df=df, top_n=10):
    reader = Reader(rating_scale=(1, 5))  # Specify the rating scale
    data = Dataset.load_from_df(df[['uniq_id', 'product_name', 'average_review_rating']], reader)
    trainset = data.build_full_trainset()
    svd_model.fit(trainset)
    # Get inner id of the product
    product_inner_id = trainset.to_inner_iid(product_name)
    # Get similar items to the product
    predictions = svd_model.test([(trainset.to_raw_uid(0), product_inner_id, 0)])
    similar_items = sorted(predictions, key=lambda x: x.est, reverse=True)[:top_n]
    # Convert inner ids to raw ids
    raw_item_ids = [trainset.to_raw_iid(prediction.iid) for prediction in similar_items]
    return raw_item_ids

svd_model = SVD()

Hybrid recommendations for 2 x MyTinyWorld Dolls House Miniature Ceramic Teapot With Olive Branchs are:
2 x MyTinyWorld Dolls House Miniature Ceramic Teapot With Olive Branchs
MyTinyWorld Dolls House Miniature Dolls House Miniature Halloween Cake
Dolls House Christmas Cake 1:12 Scale
MyTinyWorld Dolls House Miniature Classic Christmas Stocking
Dolls House Miniature Cats x2
New Dolls House Christmas Decoration Silver Xmas Tinsel
Sylvanian Families Christmas Set
Dolls House Miniature 1:12th Scale Tool Box Set
Dolls House 'At the Seaside' Wallpaper 1:12 Scale
1:12 Scale Dolls House Miniature Lighting Starter Pack


In [ ]:
#Algorithm for Random Product Recommendations

random_product_name = random.choice(df['product_name'])
recommendations = hybrid_recommendations(random_product_name, svd_model)
print("Hybrid recommendations for", random_product_name, "are:")
for i in recommendations:
  print(i)

Hybrid recommendations for HQ 1.3m Symphony Beach 2 Foil Kite are:
Brookite Cutter No.3 Kite
PLAY PARACHUTE 3.5m 12ft BY POP UP CO + BAG & 9 GAMES
Brookite Twilight Delta Kite
Brookite Skybird Kite
Brookite Dragonfly Kite
Brookite Cutter No.2 Kite
Brookite Clown Fish Windsock
HQ 1.3m Symphony Beach 2 Foil Kite
Brookite Union Jack Windsock
Brookite Fiesta Hot Air Balloon


Evaluation


In [ ]:
from surprise import accuracy
from surprise.model_selection import KFold as kf

print("Collaborative Filtering:")
rmse = accuracy.rmse(CF_predictions)
mae = accuracy.mae(CF_predictions)
final = []
for threshold in np.arange(0, 5.5, 0.5):
  tp=0
  fn=0
  fp=0
  tn=0
  temp = []
  for uid, _, true_r, est, _ in CF_predictions:
    if(true_r>=threshold):
      if(est>=threshold):
        tp = tp+1
      else:
        fn = fn+1
    else:
      if(est>=threshold):
        fp = fp+1
      else:
        tn = tn+1
    if tp == 0:
      precision = 0
      recall = 0
      f1 = 0
    else:
      precision = tp / (tp + fp)
      recall = tp / (tp + fn)
      f1 = 2 * (precision * recall) / (precision + recall)
  temp = [threshold, tp,fp,tn ,fn, precision, recall, f1]
  final.append(temp)
results = pd.DataFrame(final)
results.rename(columns={0:'threshold', 1:'tp', 2: 'fp', 3: 'tn', 4:'fn', 5: 'Precision', 6:'Recall', 7:'F1'}, inplace=True)
results

Collaborative Filtering:
RMSE: 0.3679
MAE:  0.3177


,threshold,tp,fp,tn,fn,Precision,Recall,F1
0,0.0,1997,0,0,0,1.000000,1.0,1.000000
1,0.5,1997,0,0,0,1.000000,1.0,1.000000
2,1.0,1997,0,0,0,1.000000,1.0,1.000000
3,1.5,1997,0,0,0,1.000000,1.0,1.000000
4,2.0,1997,0,0,0,1.000000,1.0,1.000000
5,2.5,1997,0,0,0,1.000000,1.0,1.000000
6,3.0,1997,0,0,0,1.000000,1.0,1.000000
7,3.5,1996,1,0,0,0.999499,1.0,0.999750
8,4.0,1994,3,0,0,0.998498,1.0,0.999248
9,4.5,1515,482,0,0,0.758638,1.0,0.862756


In [ ]:
from surprise import accuracy
from surprise.model_selection import KFold as kf

print("Matrix Factorization:")
rmse = accuracy.rmse(MF_predictions)
mae = accuracy.mae(MF_predictions)
final = []
for threshold in np.arange(0, 5.5, 0.5):
  tp=0
  fn=0
  fp=0
  tn=0
  temp = []
  for uid, _, true_r, est, _ in MF_predictions:
    if(true_r>=threshold):
      if(est>=threshold):
        tp = tp+1
      else:
        fn = fn+1
    else:
      if(est>=threshold):
        fp = fp+1
      else:
        tn = tn+1
    if tp == 0:
      precision = 0
      recall = 0
      f1 = 0
    else:
      precision = tp / (tp + fp)
      recall = tp / (tp + fn)
      f1 = 2 * (precision * recall) / (precision + recall)
  temp = [threshold, tp,fp,tn ,fn, precision, recall, f1]
  final.append(temp)
results = pd.DataFrame(final)
results.rename(columns={0:'threshold', 1:'tp', 2: 'fp', 3: 'tn', 4:'fn', 5: 'Precision', 6:'Recall', 7:'F1'}, inplace=True)
results

Matrix Factorization:
RMSE: 0.3679
MAE:  0.3177


,threshold,tp,fp,tn,fn,Precision,Recall,F1
0,0.0,1997,0,0,0,1.000000,1.0,1.000000
1,0.5,1997,0,0,0,1.000000,1.0,1.000000
2,1.0,1997,0,0,0,1.000000,1.0,1.000000
3,1.5,1997,0,0,0,1.000000,1.0,1.000000
4,2.0,1997,0,0,0,1.000000,1.0,1.000000
5,2.5,1997,0,0,0,1.000000,1.0,1.000000
6,3.0,1997,0,0,0,1.000000,1.0,1.000000
7,3.5,1996,1,0,0,0.999499,1.0,0.999750
8,4.0,1994,3,0,0,0.998498,1.0,0.999248
9,4.5,1515,482,0,0,0.758638,1.0,0.862756


In [ ]:
from sklearn.model_selection import KFold
import numpy as np
import pandas as pd
ratings_data = data.raw_ratings
ratings_df = pd.DataFrame(ratings_data, columns=['uniq_id', 'product_name', 'average_review_rating', 'timestamp'])

rating_column = ratings_df.columns[2]
kf = KFold(n_splits=5)
print(kf)

hybrid_predictions = []
for train_index, test_index in kf.split(ratings_df):
    trainset = ratings_df.iloc[train_index]
    testset = ratings_df.iloc[test_index]

    trainset_surprise = Dataset.load_from_df(trainset[['uniq_id', 'product_name', rating_column]], reader).build_full_trainset()

    svd_model.fit(trainset_surprise)
    testset_surprise = [(row['uniq_id'], row['product_name'], row[rating_column]) for index, row in testset.iterrows()]
    predictions = svd_model.test(testset_surprise)
    hybrid_predictions.extend(predictions)

print("hybrid recommendation")
rmse = accuracy.rmse(hybrid_predictions)
mae = accuracy.mae(hybrid_predictions)

final = []
for threshold in np.arange(0, 5.5, 0.5):
    tp = 0
    fn = 0
    fp = 0
    tn = 0
    temp = []
    for uid, _, true_r, est, _ in hybrid_predictions:
        if true_r >= threshold:
            if est >= threshold:
                tp += 1
            else:
                fn += 1
        else:
            if est >= threshold:
                fp += 1
            else:
                tn += 1
    if tp == 0:
        precision = 0
        recall = 0
        f1 = 0
    else:
        precision = tp / (tp + fp)
        recall = tp / (tp + fn)
        f1 = 2 * (precision * recall) / (precision + recall)
    temp = [threshold, tp, fp, tn, fn, precision, recall, f1]
    final.append(temp)

results = pd.DataFrame(final)
results.rename(columns={0: 'threshold', 1: 'tp', 2: 'fp', 3: 'tn', 4: 'fn', 5: 'Precision', 6: 'Recall', 7: 'F1'}, inplace=True)
print(results)


KFold(n_splits=5, random_state=None, shuffle=False)
hybrid recommendation
RMSE: 0.3723
MAE:  0.3211
    threshold    tp    fp    tn    fn  Precision  Recall        F1
0         0.0  9982     0     0     0   1.000000     1.0  1.000000
1         0.5  9982     0     0     0   1.000000     1.0  1.000000
2         1.0  9982     0     0     0   1.000000     1.0  1.000000
3         1.5  9982     0     0     0   1.000000     1.0  1.000000
4         2.0  9982     0     0     0   1.000000     1.0  1.000000
5         2.5  9981     1     0     0   0.999900     1.0  0.999950
6         3.0  9981     1     0     0   0.999900     1.0  0.999950
7         3.5  9976     6     0     0   0.999399     1.0  0.999699
8         4.0  9961    21     0     0   0.997896     1.0  0.998947
9         4.5  7582  2400     0     0   0.759567     1.0  0.863357
10        5.0     0     0  4842  5140   0.000000     0.0  0.000000


In [ ]:
from surprise.model_selection import cross_validate
reader = Reader(rating_scale=(1, 5))
data = Dataset.load_from_df(df[['uniq_id','product_name','average_review_rating']], reader)
svd_model = SVD()
cv_results = cross_validate(svd_model, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)
mean_rmse = cv_results['test_rmse'].mean()
mean_mae = cv_results['test_mae'].mean()
print("Mean RMSE:", mean_rmse)
print("Mean MAE:", mean_mae)

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    nan     nan     nan     nan     nan     nan     nan     
MAE (testset)     nan     nan     nan     nan     nan     nan     nan     
Fit time          0.14    0.14    0.13    0.13    0.13    0.13    0.00    
Test time         0.01    0.01    0.01    0.01    0.01    0.01    0.00    
Mean RMSE: nan
Mean MAE: nan


In [ ]:
# CNN
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Conv1D, GlobalMaxPooling1D, Dense
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import mean_squared_error, mean_absolute_error
import re

# df = pd.read_csv("Amazon_Products.csv")
df['average_review_rating'] = df['average_review_rating'].apply(lambda x: re.findall(r'\d+\.\d+', str(x))[0] if len(re.findall(r'\d+\.\d+', str(x))) > 0 else np.nan)
df.dropna(subset=['average_review_rating'], inplace=True)
df['average_review_rating'] = df['average_review_rating'].astype(float)
df['product_name'] = df['product_name'].astype(str)
le = LabelEncoder()
df['uniq_id'] = le.fit_transform(df['uniq_id'])
tokenizer = Tokenizer()
tokenizer.fit_on_texts(df['product_name'])
df['product_name_encoded'] = tokenizer.texts_to_sequences(df['product_name'])
max_seq_length = max([len(seq) for seq in df['product_name_encoded']])
X = pad_sequences(df['product_name_encoded'], maxlen=max_seq_length)
y = df['average_review_rating']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
embedding_dim = 100
vocab_size = len(tokenizer.word_index) + 1
model = Sequential()
model.add(Embedding(input_dim=vocab_size, output_dim=embedding_dim, input_length=max_seq_length))
model.add(Conv1D(128, 5, activation='relu'))
model.add(GlobalMaxPooling1D())
model.add(Dense(64, activation='relu'))
model.add(Dense(1))

model.compile(optimizer='adam', loss='mean_squared_error')

early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)
history = model.fit(X_train, y_train, epochs=20, batch_size=32, validation_split=0.1, callbacks=[early_stopping])

y_pred = model.predict(X_test).flatten()
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
mae = mean_absolute_error(y_test, y_pred)

print("RMSE:", rmse)
print("MAE:", mae)


Epoch 1/20
225/225 [==============================] - 11s 46ms/step - loss: 1.6401 - val_loss: 0.1321
Epoch 2/20
225/225 [==============================] - 4s 17ms/step - loss: 0.1128 - val_loss: 0.1348
Epoch 3/20
225/225 [==============================] - 2s 8ms/step - loss: 0.0725 - val_loss: 0.1487
Epoch 4/20
63/63 [==============================] - 0s 2ms/step
RMSE: 0.3709288083620805
MAE: 0.3282489768970473


In [ ]:
# Define a threshold for correctness (e.g., within 0.5 star rating)
threshold = 0.5

# Calculate correctness based on the threshold
correct_predictions = np.mean(np.abs(y_test - y_pred) <= threshold)

# Calculate accuracy
accuracy = correct_predictions * 100

# Print the accuracy
print("Accuracy:", accuracy, "%")

Accuracy: 85.22784176264396 %


In [ ]:
def predict_recommendations(model, tokenizer, product_name, df, top_n=10):

    product_name_encoded = tokenizer.texts_to_sequences([product_name])
    product_name_padded = pad_sequences(product_name_encoded, maxlen=max_seq_length)
    predicted_rating = model.predict(product_name_padded)[0][0]
    df['predicted_rating'] = model.predict(X).flatten()
    similar_products = df[df['predicted_rating'] >= predicted_rating].head(top_n)

    return similar_products
product_name = "Camping Site"
recommendations = predict_recommendations(model, tokenizer, product_name, df)
print("Recommended Products for:", product_name)
for index, row in recommendations.iterrows():
    print(f"Product Name: {row['product_name']}, Predicted Rating: {row['predicted_rating']}")

312/312 [==============================] - 1s 2ms/step
Recommended Products for: Camping Site
Product Name: CLASSIC TOY TRAIN SET TRACK CARRIAGES LIGHT ENGINE BOXED BOYS KIDS BATTERY, Predicted Rating: 4.612987041473389
Product Name: HORNBY Coach R4410A BR Hawksworth Corridor 3rd, Predicted Rating: 4.645163536071777
Product Name: Hornby 00 Gauge 0-4-0 Gildenlow Salt Co. Steam Locomotive Model, Predicted Rating: 4.695758819580078
Product Name: 20pcs Model Garden Light Double Heads Lamppost Scale 1:100, Predicted Rating: 4.741336822509766
Product Name: Hornby 00 Gauge 230mm BR Bogie Passenger Brake Coach Model (Red), Predicted Rating: 4.630398750305176
Product Name: Hornby Santa's Express Train Set, Predicted Rating: 4.646907806396484
Product Name: Hornby Gauge Western Express Digital Train Set with eLink and TTS Loco Train Set, Predicted Rating: 4.662628650665283
Product Name: Kato (USA) 176-1308 F3B Denver & Rio Grande Western, Predicted Rating: 4.851016521453857
Product Name: Bachmann

In [ ]:
#Algorithm for predicting the rating of the recommended products.

# Example data for testing
test_product_name = "Camping Site"

# Test the function
recommendations = predict_recommendations(model, tokenizer, test_product_name, df)

# Print recommendations
print("Recommended Products for:", test_product_name)
for index, row in recommendations.iterrows():
    print(f"Product Name: {row['product_name']}, Predicted Rating: {row['predicted_rating']}")


312/312 [==============================] - 0s 2ms/step
Recommended Products for: Camping Site
Product Name: CLASSIC TOY TRAIN SET TRACK CARRIAGES LIGHT ENGINE BOXED BOYS KIDS BATTERY, Predicted Rating: 4.612987041473389
Product Name: HORNBY Coach R4410A BR Hawksworth Corridor 3rd, Predicted Rating: 4.645163536071777
Product Name: Hornby 00 Gauge 0-4-0 Gildenlow Salt Co. Steam Locomotive Model, Predicted Rating: 4.695758819580078
Product Name: 20pcs Model Garden Light Double Heads Lamppost Scale 1:100, Predicted Rating: 4.741336822509766
Product Name: Hornby 00 Gauge 230mm BR Bogie Passenger Brake Coach Model (Red), Predicted Rating: 4.630398750305176
Product Name: Hornby Santa's Express Train Set, Predicted Rating: 4.646907806396484
Product Name: Hornby Gauge Western Express Digital Train Set with eLink and TTS Loco Train Set, Predicted Rating: 4.662628650665283
Product Name: Kato (USA) 176-1308 F3B Denver & Rio Grande Western, Predicted Rating: 4.851016521453857
Product Name: Bachmann

In [ ]:
#Accuracy
# Calculate Mean Squared Error (MSE)
mse = mean_squared_error(y_test, y_pred)

# Calculate Mean Absolute Error (MAE)
mae = mean_absolute_error(y_test, y_pred)

# Print the results
print("Mean Squared Error (MSE):", mse)
print("Mean Absolute Error (MAE):", mae)



Mean Squared Error (MSE): 0.13758818087291302
Mean Absolute Error (MAE): 0.3282489768970473
